# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

## 🔧 설정 정의하기

### 3-1. Special Token 동적 수집 및 등록
- 학습 데이터에서 실제 등장하는 모든 마스킹/발화자 토큰을 자동 수집
- `additional_special_tokens`로 등록하여 토큰이 분해되지 않도록 함
- `model.resize_token_embeddings(len(tokenizer))` 호출로 임베딩 사이즈 조정

### 3-2. 프롬프트 빌더 (발화자/turn 구조 활용)
- `build_summary_prompt()`: 요약 태스크를 명시적으로 알려주는 프롬프트 추가
- `prompt_style` 옵션: "default", "speaker_aware", "minimal", "none"
- 발화자 수에 따른 동적 프롬프트 생성 가능

### 4. 구어체 전처리 (슬랭/감탄사/의성어 정규화)
- `normalize_slang()`: ㅋㅋ→웃기다, ㅠㅠ→슬프다 등 구어체→문어체 변환
- 요약문이 문어체이므로, 모델이 직접 해석하는 것보다 미리 변환하는 게 효과적
- `SLANG_MAP`: 약 50개 이상의 슬랭/감탄사/인터넷 용어 매핑
- `normalize_repeated_chars()`: 과도한 반복 문자 정규화 (ㅋㅋㅋㅋㅋㅋ → ㅋㅋㅋㅋ)

In [2]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [ ]:
# ============================================================================
# 3-1. Special Token 동적 수집 및 등록
# ============================================================================
import re
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# 데이터 경로 정의 (커널 재시작 후에도 동작하도록)
data_path = "./data/"

# 마스킹 토큰 & 발화자 토큰 패턴 정의
MASK_PAT = r"#\w+#"      # 모든 #...# 형태의 토큰
PERSON_PAT = r"#Person\d+#"  # 발화자 토큰만
# 6-1, 6-2에서 활용할 개인정보 마스킹 패턴 (발화자 제외)
PII_MASK_PAT = r"#(?!Person\d+#)\w+#"  # #Person1# 등 제외한 마스킹 토큰

def get_mask_tokens(text):
    """텍스트에서 모든 마스킹 토큰 추출 (#Word# 형태)"""
    return re.findall(MASK_PAT, text)

def get_person_tokens(text):
    """텍스트에서 발화자 토큰만 추출 (#PersonN# 형태)"""
    return re.findall(PERSON_PAT, text)

def get_pii_mask_tokens(text):
    """텍스트에서 개인정보 마스킹 토큰만 추출 (발화자 제외)"""
    return re.findall(PII_MASK_PAT, text)

def collect_special_tokens_from_data(dialogues):
    """
    학습 데이터에서 실제 등장하는 모든 special token 수집
    - #Person1#, #Person2# 등 발화자 토큰
    - #PhoneNumber#, #Address# 등 마스킹 토큰
    """
    all_tokens = set()
    for d in dialogues:
        all_tokens.update(get_mask_tokens(d))
    return sorted(list(all_tokens))

# Train 데이터에서 special tokens 수집
train_file = os.path.join(data_path, 'train.csv')
train_df_for_tokens = pd.read_csv(train_file)
DYNAMIC_SPECIAL_TOKENS = collect_special_tokens_from_data(train_df_for_tokens['dialogue'])

print("=" * 60)
print("🔖 동적으로 수집된 Special Tokens:")
print("=" * 60)
print(f"총 {len(DYNAMIC_SPECIAL_TOKENS)}개: {DYNAMIC_SPECIAL_TOKENS}")

# ============================================================================
# 6. 정규표현식 기반 패턴 활용 전략
# ============================================================================
# 6-1. 화자 수를 명시적으로 프롬프트에 포함
# 6-2. 마스킹된 정보(#PhoneNumber# 등)를 그대로 유지하라고 지시
# ============================================================================

def analyze_dialogue_structure(dialogue_text):
    """
    대화 구조 분석: 화자 수, 마스킹 토큰 정보 추출
    
    Returns:
        dict: {
            'speakers': set of speaker tokens,
            'num_speakers': number of speakers,
            'pii_masks': set of PII masking tokens,
            'has_pii_masks': whether PII masks exist
        }
    """
    speakers = set(get_person_tokens(dialogue_text))
    pii_masks = set(get_pii_mask_tokens(dialogue_text))
    
    return {
        'speakers': speakers,
        'num_speakers': len(speakers) if speakers else 2,  # 기본값 2명
        'pii_masks': pii_masks,
        'has_pii_masks': len(pii_masks) > 0,
        'pii_mask_examples': list(pii_masks)[:3]  # 예시 최대 3개
    }

def build_speaker_aware_prompt(num_speakers):
    """
    6-1. 화자 수를 명시적으로 알려주는 프롬프트 생성
    - "한 사람 이야기만 요약하는" 버그 방지
    """
    if num_speakers == 1:
        return "다음은 1명의 독백입니다. "
    elif num_speakers == 2:
        return f"다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요. "
    else:
        return f"다음은 {num_speakers}명이 나눈 대화입니다. 각 사람의 입장을 균형 있게 반영해서 요약해 주세요. "

def build_pii_mask_instruction(pii_masks, show_examples=True):
    """
    6-2. 마스킹된 정보 유지 지시 프롬프트 생성
    - 요약문에서 엉뚱한 전화번호/주소 생성 방지
    - ROUGE 점수 향상 (레퍼런스와 동일한 마스킹 토큰 유지)
    """
    if not pii_masks:
        return ""
    
    if show_examples:
        examples = list(pii_masks)[:3]  # 최대 3개 예시
        examples_str = ", ".join(examples)
        return (
            f"대화 중 개인정보는 {examples_str} 와 같이 마스킹되어 있습니다. "
            "요약문에서도 이 마스킹 표현을 그대로 사용해 주세요. "
        )
    else:
        return (
            "대화 중 개인정보는 이미 마스킹되어 있습니다. "
            "요약문에서도 마스킹 표현을 그대로 유지해 주세요. "
        )

# 6. 설정
REGEX_PATTERN_CONFIG = {
    "use_speaker_count": True,      # 6-1. 화자 수 명시
    "use_pii_mask_instruction": True,  # 6-2. 마스킹 유지 지시
    "show_pii_examples": True,       # 마스킹 예시 표시 여부
}

print("\n" + "=" * 60)
print("🔍 6. 정규표현식 기반 패턴 활용 전략:")
print("=" * 60)
print(f"  6-1. 화자 수 명시: {REGEX_PATTERN_CONFIG['use_speaker_count']}")
print(f"  6-2. PII 마스킹 유지 지시: {REGEX_PATTERN_CONFIG['use_pii_mask_instruction']}")

# 예시 테스트
sample_analysis = analyze_dialogue_structure(train_df_for_tokens['dialogue'].iloc[0])
print(f"\n📊 대화 구조 분석 예시:")
print(f"  - 화자 수: {sample_analysis['num_speakers']}명")
print(f"  - 화자 토큰: {sample_analysis['speakers']}")
print(f"  - PII 마스킹 여부: {sample_analysis['has_pii_masks']}")
print(f"  - PII 마스킹 예시: {sample_analysis['pii_mask_examples']}")

# ============================================================================
# 4. 구어체 전처리: 슬랭/감탄사/의성어 정규화
# ============================================================================

# 슬랭/감탄사/의성어 → 문어체 매핑 사전
SLANG_MAP = {
    # 웃음 표현
    "ㅋㅋㅋㅋㅋ": "매우 웃기다",
    "ㅋㅋㅋㅋ": "매우 웃기다",
    "ㅋㅋㅋ": "웃기다",
    "ㅋㅋ": "웃기다",
    "ㅎㅎㅎㅎ": "웃기다",
    "ㅎㅎㅎ": "웃기다",
    "ㅎㅎ": "웃기다",
    "ㅋ": "",  # 단일 ㅋ는 제거
    "ㅎ": "",  # 단일 ㅎ는 제거
    
    # 슬픔/울음 표현
    "ㅠㅠㅠㅠ": "매우 슬프다",
    "ㅠㅠㅠ": "슬프다",
    "ㅠㅠ": "슬프다",
    "ㅜㅜㅜㅜ": "매우 슬프다",
    "ㅜㅜㅜ": "슬프다",
    "ㅜㅜ": "슬프다",
    "ㅠ": "",
    "ㅜ": "",
    
    # 놀람/감탄 표현
    "헐": "놀랍다",
    "헐ㅋ": "놀랍다",
    "대박": "놀랍다",
    "대박ㅋㅋ": "놀랍다",
    "와": "놀랍다",
    "와우": "놀랍다",
    "우와": "놀랍다",
    "오오": "놀랍다",
    "오오오": "매우 놀랍다",
    "ㅇㅇ": "그렇다",
    "ㄴㄴ": "아니다",
    
    # 긍정/동의 표현
    "ㄱㄱ": "가자",
    "ㄱㄱㄱ": "빨리 가자",
    "ㅇㅋ": "알겠다",
    "ㅇㅋㅇㅋ": "알겠다",
    "넹": "네",
    "넵": "네",
    "욤": "요",
    "용": "요",
    "쥬": "줘",
    
    # 부정/짜증 표현
    "아ㅋㅋ": "아",
    "에휴": "한숨",
    "ㅡㅡ": "짜증난다",
    ";;": "당황스럽다",
    ";;;": "매우 당황스럽다",
    
    # 강조 표현
    "진짜ㅋㅋ": "정말",
    "레알": "정말",
    "리얼": "정말",
    "찐": "진짜",
    "쩐다": "대단하다",
    "쩔어": "대단하다",
    "짱": "최고다",
    
    # 기타 인터넷 용어
    "ㄷㄷ": "놀랍다",
    "ㄷㄷㄷ": "매우 놀랍다",
    "ㅂㅂ": "안녕",
    "ㅃㅃ": "안녕",
    "ㄳ": "감사",
    "ㄱㅅ": "감사",
    "ㅈㅅ": "죄송",
    "ㅁㄹ": "모르겠다",
    "ㄹㅇ": "정말",
    "ㅇㄷ": "어디",
}

# 반복 문자 정규화 패턴
REPEAT_PATTERNS = [
    (r'ㅋ{5,}', 'ㅋㅋㅋㅋ'),
    (r'ㅎ{5,}', 'ㅎㅎㅎㅎ'),
    (r'ㅠ{5,}', 'ㅠㅠㅠㅠ'),
    (r'ㅜ{5,}', 'ㅜㅜㅜㅜ'),
    (r'\.{4,}', '...'),
    (r'!{3,}', '!!'),
    (r'\?{3,}', '??'),
    (r'~{3,}', '~~'),
]

def normalize_repeated_chars(text):
    """반복 문자를 정규화"""
    for pattern, replacement in REPEAT_PATTERNS:
        text = re.sub(pattern, replacement, text)
    return text

def normalize_slang(text):
    """슬랭/감탄사/의성어를 문어체로 변환"""
    text = normalize_repeated_chars(text)
    sorted_slang = sorted(SLANG_MAP.items(), key=lambda x: len(x[0]), reverse=True)
    for slang, replacement in sorted_slang:
        if slang in text:
            if replacement:
                text = text.replace(slang, f" {replacement} ")
            else:
                text = text.replace(slang, " ")
    text = ' '.join(text.split())
    return text

def preprocess_dialogue(text, normalize_slang_flag=True):
    """대화 전처리 파이프라인"""
    if not text or not isinstance(text, str):
        return text
    if normalize_slang_flag:
        text = normalize_slang(text)
    return text

PREPROCESS_CONFIG = {
    "normalize_slang": True,
}

print("\n" + "=" * 60)
print("🗣️ 구어체 전처리 (슬랭/감탄사 정규화):")
print("=" * 60)
print(f"총 {len(SLANG_MAP)}개 슬랭 매핑 정의")

# ============================================================================
# 5. TF-IDF 키워드 추출 및 프롬프트 활용
# ============================================================================
# (sklearn은 셀 상단에서 이미 import됨)

# TF-IDF 설정
TFIDF_CONFIG = {
    "min_df": 2,           # 최소 문서 빈도
    "max_df": 0.9,         # 최대 문서 빈도 (90% 이상 등장 단어 제외)
    "max_features": 2000,  # 최대 feature 수
    "ngram_range": (1, 2), # unigram + bigram
    "top_k": 5,            # 추출할 키워드 개수
}

print("\n" + "=" * 60)
print("📊 TF-IDF 설정:")
print("=" * 60)
print(f"  min_df: {TFIDF_CONFIG['min_df']}")
print(f"  max_df: {TFIDF_CONFIG['max_df']}")
print(f"  max_features: {TFIDF_CONFIG['max_features']}")
print(f"  ngram_range: {TFIDF_CONFIG['ngram_range']}")
print(f"  top_k: {TFIDF_CONFIG['top_k']}")

# TF-IDF Vectorizer 초기화 (한국어 불용어 포함)
KOREAN_STOPWORDS = [
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '로', '으로', '와', '과',
    '도', '만', '까지', '부터', '이다', '있다', '하다', '되다', '수', '것', '등',
    '그', '저', '이런', '저런', '그런', '어떤', '무슨', '뭐', '뭔가', '뭘',
    '나', '너', '우리', '저희', '그녀', '그', '이것', '저것', '그것',
    '아', '어', '오', '예', '네', '응', '음', '엄', '아니', '안',
    '좀', '잘', '더', '덜', '많이', '조금', '아주', '매우', '정말', '진짜',
    '그래서', '그러면', '그런데', '하지만', '그리고', '또', '또한',
    '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#',
]

class TFIDFKeywordExtractor:
    """TF-IDF 기반 키워드 추출기"""
    
    def __init__(self, corpus=None, max_features=1000, ngram_range=(1, 2)):
        """
        Args:
            corpus: TF-IDF 학습용 코퍼스 (대화 리스트)
            max_features: 최대 feature 수
            ngram_range: n-gram 범위
        """
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,
            ngram_range=ngram_range,
            token_pattern=r'[가-힣]+',  # 한글만 추출
            stop_words=KOREAN_STOPWORDS,
        )
        self.is_fitted = False
        
        if corpus is not None:
            self.fit(corpus)
    
    def fit(self, corpus):
        """코퍼스로 TF-IDF 학습"""
        # 마스킹 토큰 제거 후 학습
        cleaned_corpus = [re.sub(MASK_PAT, ' ', doc) for doc in corpus]
        self.vectorizer.fit(cleaned_corpus)
        self.feature_names = self.vectorizer.get_feature_names_out()
        self.is_fitted = True
        print(f"✅ TF-IDF 학습 완료: {len(self.feature_names)}개 feature")
    
    def extract_keywords(self, text, top_k=5):
        """
        단일 문서에서 TF-IDF 상위 키워드 추출
        
        Args:
            text: 대화 텍스트
            top_k: 추출할 키워드 수
        
        Returns:
            List of (keyword, score) tuples
        """
        if not self.is_fitted:
            return []
        
        # 마스킹 토큰 제거
        cleaned_text = re.sub(MASK_PAT, ' ', text)
        
        try:
            tfidf_vector = self.vectorizer.transform([cleaned_text])
            scores = tfidf_vector.toarray()[0]
            
            # 상위 k개 인덱스
            top_indices = scores.argsort()[-top_k:][::-1]
            
            keywords = []
            for idx in top_indices:
                if scores[idx] > 0:
                    keywords.append((self.feature_names[idx], scores[idx]))
            
            return keywords
        except:
            return []
    
    def get_keyword_string(self, text, top_k=5, separator=" / "):
        """키워드를 문자열로 반환"""
        keywords = self.extract_keywords(text, top_k)
        if not keywords:
            return ""
        return separator.join([kw for kw, score in keywords])

# TF-IDF 추출기 초기화 (학습 데이터 기반)
print("\n" + "=" * 60)
print("📊 TF-IDF 키워드 추출기 초기화:")
print("=" * 60)

tfidf_extractor = TFIDFKeywordExtractor(
    corpus=train_df_for_tokens['dialogue'].tolist(),
    max_features=2000,
    ngram_range=(1, 2)
)

# 예시 테스트
sample_text = train_df_for_tokens['dialogue'].iloc[0]
sample_keywords = tfidf_extractor.get_keyword_string(sample_text, top_k=5)
print(f"\n📝 TF-IDF 키워드 추출 예시:")
print(f"  키워드: {sample_keywords}")

# ============================================================================
# 5-1. TF-IDF 키워드 기반 프롬프트 빌더 (6-1, 6-2 통합)
# ============================================================================

def build_tfidf_keyword_prompt(dialogue_text, tfidf_extractor, top_k=5, prompt_style="keyword",
                                use_speaker_count=True, use_pii_instruction=True):
    """
    TF-IDF 키워드를 포함한 프롬프트 생성 (6-1, 6-2 통합)
    
    Args:
        dialogue_text: 원본 대화 텍스트
        tfidf_extractor: TFIDFKeywordExtractor 인스턴스
        top_k: 추출할 키워드 수
        prompt_style: "keyword" (키워드만), "keyword_aware" (키워드+발화자수)
        use_speaker_count: 6-1. 화자 수 명시 여부
        use_pii_instruction: 6-2. PII 마스킹 유지 지시 여부
    
    Returns:
        프롬프트가 추가된 대화 텍스트
    """
    # 대화 구조 분석
    analysis = analyze_dialogue_structure(dialogue_text)
    num_speakers = analysis['num_speakers']
    pii_masks = analysis['pii_masks']
    
    # 키워드 추출
    keywords = tfidf_extractor.get_keyword_string(dialogue_text, top_k=top_k)
    
    prompt_parts = []
    
    # 6-1. 화자 수 명시 (keyword_aware 스타일에서만)
    if prompt_style == "keyword_aware" and use_speaker_count:
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "keyword":
        prompt_parts.append("다음 대화를 한 문단의 자연스러운 문어체 한국어로 요약해 주세요. ")
    
    # TF-IDF 키워드 추가
    if keywords:
        prompt_parts.append(f"주요 키워드: {keywords}. ")
    
    # 6-2. PII 마스킹 유지 지시
    if use_pii_instruction and pii_masks:
        prompt_parts.append(build_pii_mask_instruction(pii_masks, show_examples=REGEX_PATTERN_CONFIG['show_pii_examples']))
    
    prompt_parts.append("\n\n")
    
    prompt = "".join(prompt_parts)
    return prompt + dialogue_text

# ============================================================================
# 5-2. 대화 길이/주제별 동적 추론 설정
# ============================================================================

# 길이 기반 추론 설정
LENGTH_BASED_INFERENCE_CONFIG = {
    # 짧은 대화 (500자 이하)
    "short": {
        "char_threshold": 500,
        "max_new_tokens": 60,
        "num_beams": 4,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    },
    # 중간 대화 (500~1500자)
    "medium": {
        "char_threshold": 1500,
        "max_new_tokens": 80,
        "num_beams": 5,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    },
    # 긴 대화 (1500자 초과)
    "long": {
        "char_threshold": float('inf'),
        "max_new_tokens": 100,
        "num_beams": 6,
        "length_penalty": 1.2,  # 긴 대화는 요약도 길게
        "no_repeat_ngram_size": 3,
    },
}

# 주제 키워드 기반 추론 설정 (특정 주제는 더 긴 요약 필요)
TOPIC_KEYWORDS = {
    "medical": ["병원", "의사", "치료", "진료", "수술", "약", "증상", "진단", "검사"],
    "insurance": ["보험", "가입", "보장", "청구", "계약", "납입", "보상", "약관"],
    "finance": ["대출", "이자", "금리", "계좌", "송금", "입금", "출금", "투자", "주식"],
    "travel": ["여행", "비행기", "호텔", "예약", "관광", "숙소", "항공"],
}

TOPIC_INFERENCE_CONFIG = {
    "medical": {"max_new_tokens": 100, "length_penalty": 1.3},
    "insurance": {"max_new_tokens": 100, "length_penalty": 1.3},
    "finance": {"max_new_tokens": 95, "length_penalty": 1.2},
    "travel": {"max_new_tokens": 85, "length_penalty": 1.1},
    "default": {"max_new_tokens": 80, "length_penalty": 1.0},
}

def detect_topic(dialogue_text):
    """대화 주제 감지"""
    text_lower = dialogue_text.lower()
    
    topic_scores = {}
    for topic, keywords in TOPIC_KEYWORDS.items():
        score = sum(1 for kw in keywords if kw in text_lower)
        if score > 0:
            topic_scores[topic] = score
    
    if topic_scores:
        return max(topic_scores, key=topic_scores.get)
    return "default"

def get_length_category(dialogue_text):
    """대화 길이 카테고리 반환"""
    length = len(dialogue_text)
    
    if length <= LENGTH_BASED_INFERENCE_CONFIG["short"]["char_threshold"]:
        return "short"
    elif length <= LENGTH_BASED_INFERENCE_CONFIG["medium"]["char_threshold"]:
        return "medium"
    else:
        return "long"

def get_dynamic_inference_config(dialogue_text, use_topic=True, use_length=True):
    """
    대화 특성에 따른 동적 추론 설정 반환
    
    Args:
        dialogue_text: 대화 텍스트
        use_topic: 주제 기반 설정 사용 여부
        use_length: 길이 기반 설정 사용 여부
    
    Returns:
        dict: 추론 설정 (max_new_tokens, num_beams, length_penalty 등)
    """
    config = {
        "max_new_tokens": 80,
        "num_beams": 5,
        "length_penalty": 1.0,
        "no_repeat_ngram_size": 2,
    }
    
    # 1. 길이 기반 설정
    if use_length:
        length_cat = get_length_category(dialogue_text)
        length_config = LENGTH_BASED_INFERENCE_CONFIG[length_cat]
        config.update({
            "max_new_tokens": length_config["max_new_tokens"],
            "num_beams": length_config["num_beams"],
            "length_penalty": length_config["length_penalty"],
            "no_repeat_ngram_size": length_config["no_repeat_ngram_size"],
        })
    
    # 2. 주제 기반 설정 (길이보다 우선)
    if use_topic:
        topic = detect_topic(dialogue_text)
        if topic != "default":
            topic_config = TOPIC_INFERENCE_CONFIG[topic]
            config["max_new_tokens"] = max(config["max_new_tokens"], topic_config["max_new_tokens"])
            config["length_penalty"] = max(config["length_penalty"], topic_config["length_penalty"])
    
    return config

print("\n" + "=" * 60)
print("📏 길이/주제 기반 동적 추론 설정:")
print("=" * 60)
print("길이 카테고리:")
for cat, cfg in LENGTH_BASED_INFERENCE_CONFIG.items():
    print(f"  - {cat}: ~{cfg['char_threshold']}자, max_tokens={cfg['max_new_tokens']}, beams={cfg['num_beams']}")

print("\n주제 카테고리:")
for topic in TOPIC_KEYWORDS.keys():
    cfg = TOPIC_INFERENCE_CONFIG[topic]
    print(f"  - {topic}: max_tokens={cfg['max_new_tokens']}, length_penalty={cfg['length_penalty']}")

# ============================================================================
# 3-2. 프롬프트 빌더 함수 - 발화자/turn 구조 활용 (6-1, 6-2 통합)
# ============================================================================

def build_summary_prompt(dialogue_text, prompt_style="default", tfidf_extractor=None, top_k=5,
                         use_speaker_count=True, use_pii_instruction=True):
    """
    대화에 요약 프롬프트를 추가하여 모델에게 태스크를 명시적으로 알림
    6-1. 화자 수 명시: "한 사람 이야기만 요약하는" 버그 방지
    6-2. PII 마스킹 유지 지시: ROUGE 점수 향상
    
    Args:
        dialogue_text: 원본 대화 텍스트
        prompt_style: 프롬프트 스타일
            - "default": 기본 프롬프트
            - "speaker_aware": 발화자 수 포함
            - "minimal": 최소 프롬프트
            - "keyword": TF-IDF 키워드 포함 (5-1)
            - "keyword_aware": 키워드 + 발화자 수 (5-1)
            - "balanced": 6-1, 6-2 전략 적용 (NEW)
            - "none": 프롬프트 없음
        tfidf_extractor: TFIDFKeywordExtractor 인스턴스 (keyword 스타일용)
        top_k: 키워드 추출 개수
        use_speaker_count: 6-1. 화자 수 명시 여부
        use_pii_instruction: 6-2. PII 마스킹 유지 지시 여부
    
    Returns:
        프롬프트가 추가된 대화 텍스트
    """
    if prompt_style == "none":
        return dialogue_text
    
    # 대화 구조 분석 (6-1, 6-2용)
    analysis = analyze_dialogue_structure(dialogue_text)
    num_speakers = analysis['num_speakers']
    pii_masks = analysis['pii_masks']
    
    # 5-1. TF-IDF 키워드 기반 프롬프트 (6-1, 6-2 통합)
    if prompt_style in ["keyword", "keyword_aware"] and tfidf_extractor is not None:
        return build_tfidf_keyword_prompt(
            dialogue_text, tfidf_extractor, top_k, prompt_style,
            use_speaker_count=use_speaker_count,
            use_pii_instruction=use_pii_instruction
        )
    
    prompt_parts = []
    
    # 6-1. 화자 수 명시 프롬프트
    if prompt_style == "balanced":
        # 새로운 "balanced" 스타일: 6-1, 6-2 전략 모두 적용
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "speaker_aware":
        prompt_parts.append(build_speaker_aware_prompt(num_speakers))
    elif prompt_style == "minimal":
        prompt_parts.append("다음 대화를 요약하세요. ")
    else:  # default
        if use_speaker_count:
            prompt_parts.append(build_speaker_aware_prompt(num_speakers))
        else:
            prompt_parts.append("다음은 여러 사람이 나눈 일상 대화입니다. 이 대화를 한 문단의 자연스러운 문어체 한국어로 요약해 주세요. ")
    
    # 6-2. PII 마스킹 유지 지시 (balanced, default, speaker_aware 스타일에서)
    if use_pii_instruction and pii_masks and prompt_style in ["balanced", "default", "speaker_aware"]:
        prompt_parts.append(build_pii_mask_instruction(pii_masks, show_examples=REGEX_PATTERN_CONFIG['show_pii_examples']))
    
    prompt_parts.append("\n\n")
    
    prompt = "".join(prompt_parts)
    return prompt + dialogue_text

def add_turn_separators(dialogue_text, separator="[SEP]"):
    """발화 turn 사이에 구분자 추가"""
    lines = dialogue_text.strip().split('\n')
    result_lines = []
    for i, line in enumerate(lines):
        if i > 0 and re.match(PERSON_PAT, line.strip()):
            result_lines.append(f"{separator} {line}")
        else:
            result_lines.append(line)
    return '\n'.join(result_lines)

# ============================================================================
# 설정 업데이트 (6. 정규표현식 패턴 활용 전략 반영)
# ============================================================================

config_data = {
    "general": {
        "data_path": "./data/",
        "model_name": "digit82/kobart-summarization",
        "output_dir": "./prediction_kobart_v1"
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": DYNAMIC_SPECIAL_TOKENS
    },
    # 3-2. 프롬프트 설정 (6-1, 6-2 통합)
    "prompt": {
        "use_prompt": True,
        "prompt_style": "balanced",  # NEW: 6-1, 6-2 전략 적용
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,  # 5-1. TF-IDF 키워드 개수
    },
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    "regex_pattern": {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    },
    # 4. 구어체 전처리 설정
    "preprocess": {
        "normalize_slang": True,
    },
    # 5-2. 동적 추론 설정
    "dynamic_inference": {
        "use_length_based": True,   # 길이 기반 동적 설정 사용
        "use_topic_based": True,    # 주제 기반 동적 설정 사용
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "eval_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"
    },
    "wandb": {
        "entity": "fc_bootcamp",
        "project": "kobart_v1",
        "name": "baseline_v0"
    },
    "inference": {
        "ckt_path": "model ckt path",
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

# ============================================================================
# 설정 요약 출력
# ============================================================================

print("\n" + "=" * 60)
print("📋 전체 설정 요약:")
print("=" * 60)

print("\n🔖 3-1. Special Tokens:")
print(f"  - 동적 수집: {len(DYNAMIC_SPECIAL_TOKENS)}개")

print("\n📝 3-2. 프롬프트 설정:")
print(f"  - 프롬프트 사용: {config_data['prompt']['use_prompt']}")
print(f"  - 프롬프트 스타일: {config_data['prompt']['prompt_style']}")
print(f"  - TF-IDF 키워드 수: {config_data['prompt']['tfidf_top_k']}개")

print("\n🔍 6. 정규표현식 패턴 활용 전략:")
print(f"  - 6-1. 화자 수 명시: {config_data['regex_pattern']['use_speaker_count']}")
print(f"  - 6-2. PII 마스킹 유지 지시: {config_data['regex_pattern']['use_pii_instruction']}")

print("\n🗣️ 4. 구어체 전처리:")
print(f"  - 슬랭 정규화: {config_data['preprocess']['normalize_slang']}")

print("\n📏 5-2. 동적 추론:")
print(f"  - 길이 기반: {config_data['dynamic_inference']['use_length_based']}")
print(f"  - 주제 기반: {config_data['dynamic_inference']['use_topic_based']}")

# ============================================================================
# 6. 전처리 파이프라인 예시 (6-1, 6-2 통합)
# ============================================================================
print("\n" + "-" * 60)
print("📝 전처리 파이프라인 예시 (6-1, 6-2 적용):")
print("-" * 60)

sample_dialogue = train_df_for_tokens['dialogue'].iloc[0]
print(f"[1. 원본 대화 (앞 150자)]:\n{sample_dialogue[:150]}...")

# 대화 구조 분석
sample_analysis = analyze_dialogue_structure(sample_dialogue)
print(f"\n[2. 대화 구조 분석]:")
print(f"  - 화자 수: {sample_analysis['num_speakers']}명")
print(f"  - PII 마스킹 토큰: {sample_analysis['pii_mask_examples']}")

sample_preprocessed = preprocess_dialogue(sample_dialogue, normalize_slang_flag=True)
print(f"\n[3. 슬랭 정규화 후]:\n{sample_preprocessed[:150]}...")

sample_keywords = tfidf_extractor.get_keyword_string(sample_preprocessed, top_k=5)
print(f"\n[4. TF-IDF 키워드]: {sample_keywords}")

# 6-1, 6-2 통합 프롬프트 (balanced 스타일)
sample_prompted = build_summary_prompt(
    sample_preprocessed, 
    prompt_style="balanced",
    use_speaker_count=True,
    use_pii_instruction=True
)
print(f"\n[5. 'balanced' 프롬프트 적용 (6-1, 6-2)]:\n{sample_prompted[:400]}...")

# keyword_aware 스타일 예시
sample_keyword_aware = build_summary_prompt(
    sample_preprocessed,
    prompt_style="keyword_aware",
    tfidf_extractor=tfidf_extractor,
    top_k=5,
    use_speaker_count=True,
    use_pii_instruction=True
)
print(f"\n[6. 'keyword_aware' 프롬프트 적용 (TF-IDF + 6-1, 6-2)]:\n{sample_keyword_aware[:400]}...")

# 동적 추론 설정 예시
dynamic_config = get_dynamic_inference_config(sample_dialogue, use_topic=True, use_length=True)
print(f"\n[7. 동적 추론 설정]:")
print(f"  - 길이 카테고리: {get_length_category(sample_dialogue)}")
print(f"  - 감지된 주제: {detect_topic(sample_dialogue)}")
print(f"  - max_new_tokens: {dynamic_config['max_new_tokens']}")

# ============================================================================
# 유틸리티 함수: 디코딩 & 후처리
# ============================================================================

def decode_with_speaker_tags(tokenizer, output_ids, remove_tokens=None):
    """
    화자 태그(#Person1# 등)를 보존하면서 디코딩
    skip_special_tokens=False로 디코딩 후, 불필요한 토큰만 제거
    """
    if remove_tokens is None:
        remove_tokens = ['<usr>', tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token]
    
    # skip_special_tokens=False로 디코딩 (화자 태그 보존)
    text = tokenizer.decode(output_ids, skip_special_tokens=False)
    
    # 불필요한 토큰만 제거
    for token in remove_tokens:
        if token:
            text = text.replace(token, '')
    
    return text.strip()

def postprocess_summary(summary):
    """
    요약문 후처리:
    1. 불필요한 공백 제거
    2. 화자 태그 앞뒤 공백 정리
    """
    import re
    
    # 연속 공백 제거
    summary = ' '.join(summary.split())
    
    # 화자 태그 앞뒤 공백 정리 (#Person1# 은 → #Person1#은)
    summary = re.sub(r'(#Person\d+#)\s+', r'\1 ', summary)
    
    return summary.strip()

print("\n✅ decode_with_speaker_tags, postprocess_summary 함수 정의 완료")

🔖 동적으로 수집된 Special Tokens:
총 22개: ['#Address#', '#Alex#', '#Bob#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#Kristin#', '#Liliana#', '#Name#', '#PassportNumber#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PersonName#', '#PhoneNumber#', '#Price#', '#SSN#']

🔍 6. 정규표현식 기반 패턴 활용 전략:
  6-1. 화자 수 명시: True
  6-2. PII 마스킹 유지 지시: True

📊 대화 구조 분석 예시:
  - 화자 수: 2명
  - 화자 토큰: {'#Person1#', '#Person2#'}
  - PII 마스킹 여부: False
  - PII 마스킹 예시: []

🗣️ 구어체 전처리 (슬랭/감탄사 정규화):
총 59개 슬랭 매핑 정의

📊 TF-IDF 키워드 추출기 초기화:
✅ TF-IDF 학습 완료: 2000개 feature

📝 TF-IDF 키워드 추출 예시:
  키워드: 매년 / 한 / 번 / 해요 / 의사 선생님

📏 길이/주제 기반 동적 추론 설정:
길이 카테고리:
  - short: ~500자, max_tokens=60, beams=4
  - medium: ~1500자, max_tokens=80, beams=5
  - long: ~inf자, max_tokens=100, beams=6

주제 카테고리:
  - medical: max_tokens=100, length_penalty=1.3
  - insurance: max_tokens=100, length_penalty=1.3
  - finance: max_tokens=95, length_penalty=1.2
  - travel: max_tokens=85, length_p

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [4]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [5]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'dynamic_inference': {'use_length_based': True, 'use_topic_based': True},
 'general': {'data_path': './data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './prediction_kobart_v1'},
 'inference': {'batch_size': 32,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'preprocess': {'normalize_slang': True},
 'prompt': {'prompt_style': 'balanced',
            'tfidf_top_k': 5,
            'turn_separator': '[SEP]',
            'use_prompt': True,
            'use_turn_separator': False},
 'regex_pattern': {'show_pii_examples': True,
                   'use_pii_instruction': True,
                   'use_speaker_count': True},
 'tokenizer': {'bos_token': '<s>',
               'decoder_ma

In [6]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': './data/',
 'model_name': 'digit82/kobart-summarization',
 'output_dir': './prediction_kobart_v1'}

In [7]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

# Ensure compatibility with Transformers: they expect the key to be 'evaluation_strategy'.
# Some configs may use 'eval_strategy' (short form). Normalize to 'evaluation_strategy'.
if 'eval_strategy' in loaded_config['training'] and 'evaluation_strategy' not in loaded_config['training']:
    loaded_config['training']['evaluation_strategy'] = loaded_config['training'].pop('eval_strategy')

# (legacy) if someone used 'evaluation_strategy' already, keep it as-is.


In [8]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 100,
 'encoder_max_len': 1024,
 'eos_token': '</s>',
 'special_tokens': ['#Address#',
  '#Alex#',
  '#Bob#',
  '#CarNumber#',
  '#CardNumber#',
  '#DateOfBirth#',
  '#Email#',
  '#Kristin#',
  '#Liliana#',
  '#Name#',
  '#PassportNumber#',
  '#Person1#',
  '#Person2#',
  '#Person3#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#',
  '#PersonName#',
  '#PhoneNumber#',
  '#Price#',
  '#SSN#']}

In [9]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 2e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 16,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01,
 'evaluation_strategy': 'epoch'}

In [10]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'fc_bootcamp', 'name': 'baseline_v0', 'project': 'kobart_v1'}

In [11]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "fc_bootcamp"
loaded_config['wandb']['project'] = "kobart_v1"
loaded_config['wandb']['name'] = "baseline_v0"

# wandb 비활성화 (권한 문제 해결)
loaded_config['training']['report_to'] = "wandb"

In [12]:
# --- Wandb setup helper (추가) ---
# 사용법 예시:
# cfg = setup_wandb(api_key="<YOUR_API_KEY>", entity="your-entity", project="your-project", run_name="exp-01")
# import wandb
# wandb.init(**cfg)

import os

def setup_wandb(api_key: str = None, entity: str = None, project: str = None, run_name: str = None, mode: str = 'online'):
    """
    Helper to set WANDB API key and login programmatically.

    - api_key: your W&B API key (string). If provided, sets `WANDB_API_KEY` env var.
    - entity: W&B team or user name (maps to `entity` in `wandb.init`).
    - project: W&B project name (maps to `project` in `wandb.init`).
    - run_name: a human-friendly run name (maps to `name` in `wandb.init`).
    - mode: 'online' or 'offline' (useful when running without internet or CI).

    Returns a dict that can be passed to `wandb.init(**cfg)` or used to populate `loaded_config['wandb']`.
    """
    if api_key:
        os.environ['WANDB_API_KEY'] = '624410f236117b79b2ee07a4ccef31c7514a5e03'

    try:
        import wandb
    except Exception as e:
        raise ImportError("wandb is not installed. Install with `pip install wandb` and try again.") from e

    env_key = os.environ.get('WANDB_API_KEY')
    if env_key is None:
        print("Warning: No WANDB_API_KEY found in environment. Call setup_wandb(api_key=...) or set WANDB_API_KEY manually.")

    # Perform login (this will use the env var if present)
    try:
        wandb.login(key=env_key)
    except Exception as e:
        print("wandb.login() failed:", e)

    cfg = {
        'mode': mode
    }
    if entity:
        cfg['entity'] = fc_bootcamp
    if project:
        cfg['project'] = kobart_v1
    if run_name:
        cfg['name'] = baseline_v0

    print("W&B helper ready. Use: import wandb; wandb.init(**cfg)")
    return cfg

# 예시 (주석 해제 후 사용):
# cfg = setup_wandb(api_key="<YOUR_API_KEY>", entity="your-team-or-username", project="my-project", run_name="run-1")
# import wandb
# wandb.init(**cfg)
# ----------------------------------


In [13]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
 'result_path': './prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [14]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,호텔 안내
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,회의 준비
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,차량 대여
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,실직과 대처
12456,train_12459,"#Person1#: 엄마, 나 다음 주 토요일에 이모부네 가족 보러 가는데, 오늘 ...",#Person1#은 다음 주 토요일에 이모부네 가족을 방문하기 위해 짐을 싸야 하는...,가족 방문 준비


In [15]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 새해가 되니까 나도 새 출발을 하기로 했어.\n#Person2#...,#Person1#은 새해에 담배를 끊고 커밍아웃 하기로 결심했습니다. #Person...,새해 결심
495,dev_496,#Person1#: 너 Joe랑 결혼했지?\n#Person2#: Joe? 무슨 말이...,"#Person1#은 #Person2#가 Joe와 결혼했다고 생각하지만, #Perso...",사랑과 결혼 오해
496,dev_497,"#Person1#: 어떻게 도와드릴까요, 아줌마?\n#Person2#: 제 차에서 ...","#Person2#의 차에서 소리가 나며, 브레이크 수리가 필요한 상황입니다. #Pe...",차량 소음 및 수리
497,dev_498,"#Person1#: 여보세요, 아마존 고객 서비스입니다. 어떻게 도와드릴까요?\n#...",#Person2#가 아마존 고객 서비스에 전화하여 아마존에서 구매한 책에 53페이지...,책 페이지 누락
498,dev_499,#Person1#: 벌써 여름이 다가오다니 믿기지 않아. \n#Person2#: 맞...,"#Person2#는 여름방학 동안 파티에서 일하는 회사에서 일하며, 주로 음식 준비...",여름방학 일자리


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [16]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
            prompt_config: dict = None,           # 3-2. 프롬프트 설정
            preprocess_config: dict = None,       # 4. 구어체 전처리 설정
            tfidf_extractor = None,               # 5-1. TF-IDF 키워드 추출기
            regex_pattern_config: dict = None,    # 6. 정규표현식 패턴 활용 설정 (NEW)
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token
        
        # 프롬프트 설정 저장
        self.prompt_config = prompt_config or {
            "use_prompt": False,
            "prompt_style": "none",
            "use_turn_separator": False,
            "turn_separator": "[SEP]",
            "tfidf_top_k": 5,
        }
        
        # 4. 구어체 전처리 설정 저장
        self.preprocess_config = preprocess_config or {
            "normalize_slang": False,
        }
        
        # 5-1. TF-IDF 키워드 추출기
        self.tfidf_extractor = tfidf_extractor
        
        # 6. 정규표현식 패턴 활용 설정 (NEW)
        self.regex_pattern_config = regex_pattern_config or {
            "use_speaker_count": True,       # 6-1. 화자 수 명시
            "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
            "show_pii_examples": True,       # 마스킹 예시 표시
        }

    @staticmethod
    def make_set_as_df(file_path, is_train=True):
        """실험에 필요한 컬럼을 가져옵니다."""
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname', 'dialogue']]
            return test_df

    def apply_slang_normalization(self, text):
        """4. 구어체 전처리: 슬랭/감탄사/의성어 정규화"""
        if self.preprocess_config.get("normalize_slang", False):
            return preprocess_dialogue(text, normalize_slang_flag=True)
        return text

    def apply_prompt_to_dialogue(self, dialogue_text):
        """
        3-2, 5-1, 6-1, 6-2 통합 프롬프트 적용
        
        - 3-2: 프롬프트 스타일 적용
        - 5-1: TF-IDF 키워드 포함
        - 6-1: 화자 수 명시 (균형 있는 요약 유도)
        - 6-2: PII 마스킹 유지 지시 (ROUGE 점수 향상)
        """
        text = dialogue_text
        
        # 발화 구분자 적용 (선택적)
        if self.prompt_config.get("use_turn_separator", False):
            text = add_turn_separators(text, self.prompt_config.get("turn_separator", "[SEP]"))
        
        # 프롬프트 적용
        if self.prompt_config.get("use_prompt", False):
            prompt_style = self.prompt_config.get("prompt_style", "default")
            tfidf_top_k = self.prompt_config.get("tfidf_top_k", 5)
            
            # 6-1, 6-2 설정 가져오기
            use_speaker_count = self.regex_pattern_config.get("use_speaker_count", True)
            use_pii_instruction = self.regex_pattern_config.get("use_pii_instruction", True)
            
            # 5-1. TF-IDF 키워드 스타일인 경우 extractor 전달
            if prompt_style in ["keyword", "keyword_aware"] and self.tfidf_extractor is not None:
                text = build_summary_prompt(
                    text, 
                    prompt_style=prompt_style, 
                    tfidf_extractor=self.tfidf_extractor, 
                    top_k=tfidf_top_k,
                    use_speaker_count=use_speaker_count,      # 6-1
                    use_pii_instruction=use_pii_instruction    # 6-2
                )
            else:
                # 기타 스타일에서도 6-1, 6-2 적용
                text = build_summary_prompt(
                    text, 
                    prompt_style=prompt_style,
                    use_speaker_count=use_speaker_count,      # 6-1
                    use_pii_instruction=use_pii_instruction    # 6-2
                )
        
        return text

    def full_preprocess_pipeline(self, dialogue_text):
        """
        전체 전처리 파이프라인: 슬랭 정규화 → 프롬프트 적용
        
        처리 순서:
        1. 슬랭/감탄사/의성어 정규화 (4. 구어체 전처리)
        2. 발화 구분자 추가 (선택적)
        3. 프롬프트 추가 (3-2. 프롬프트 설정)
           - TF-IDF 키워드 포함 (5-1)
           - 화자 수 명시 (6-1)
           - PII 마스킹 유지 지시 (6-2)
        """
        text = dialogue_text
        
        # Step 1: 슬랭 정규화
        text = self.apply_slang_normalization(text)
        
        # Step 2 & 3: 프롬프트 적용 (구분자 + TF-IDF + 6-1 + 6-2)
        text = self.apply_prompt_to_dialogue(text)
        
        return text

    def make_input(self, dataset, is_test=False, tokenizer=None, encoder_max_len=None, 
                   prompt=None, apply_dynamic_prompt=True):
        """
        BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
        
        전처리 파이프라인:
        1. 슬랭/감탄사 정규화 (4. 구어체 전처리)
        2. TF-IDF 키워드 추출 (5-1)
        3. 프롬프트 추가 (3-2. 프롬프트 설정)
        4. 6-1 화자 수 명시 + 6-2 PII 마스킹 유지 지시
        5. 토큰 기반 truncation (선택적)
        """
        if is_test:
            encoder_input = []
            decoder_input = []
            for d in dataset['dialogue']:
                text = d
                
                # 전체 전처리 파이프라인 적용
                if apply_dynamic_prompt:
                    text = self.full_preprocess_pipeline(text)
                elif prompt:
                    text = self.apply_slang_normalization(text)
                    text = apply_summary_prompt(text, prompt)
                else:
                    text = self.apply_slang_normalization(text)
                
                # 토큰 기반 truncation
                if tokenizer is not None and encoder_max_len is not None:
                    text = smart_truncate_dialogue(text, tokenizer, encoder_max_len)
                
                encoder_input.append(text)
                decoder_input.append(self.bos_token)
            return encoder_input, decoder_input
        else:
            # 학습 데이터 처리
            encoder_input = []
            for d in dataset['dialogue']:
                text = d
                
                # 전체 전처리 파이프라인 적용
                if apply_dynamic_prompt:
                    text = self.full_preprocess_pipeline(text)
                else:
                    text = self.apply_slang_normalization(text)
                
                encoder_input.append(text)
            
            # Gold target을 디코더 입력으로 사용합니다.
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            return encoder_input, decoder_input.tolist(), decoder_output.tolist()

In [17]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [18]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    """
    3-2 개선: 프롬프트 설정이 적용된 데이터셋 준비
    """
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'[원본] train_data dialogue:\n {train_data["dialogue"][0][:200]}...')
    print(f'train_label:\n {train_data["summary"][0]}')

    # 3-2. 프롬프트 적용된 데이터로 변환
    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(
        train_data, 
        apply_dynamic_prompt=True  # 프롬프트 설정 사용
    )
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(
        val_data,
        apply_dynamic_prompt=True
    )
    
    # 프롬프트 적용 확인
    print('-'*150)
    print(f'[프롬프트 적용 후] encoder_input_train[0]:\n {encoder_input_train[0][:300]}...')
    print('-'*10, 'Load data complete', '-'*10)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10)
    
    # 토큰화 후 special token 확인
    print("\n📊 토큰화 결과 확인:")
    sample_tokens = tokenized_encoder_inputs['input_ids'][0][:50]
    decoded_sample = tokenizer.decode(sample_tokens)
    print(f"   첫 50 토큰 디코딩: {decoded_sample}...")
    
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [19]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [20]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result


# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training'].get('evaluation_strategy', 'epoch'), # transformers expects this key
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) wandb 사용 시에만 초기화
    if config['training']['report_to'] == 'wandb':
        wandb.init(
            entity=config['wandb']['entity'],
            project=config['wandb']['project'],
            name=config['wandb']['name'],
        )
        # 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
        os.environ["WANDB_LOG_MODEL"]="true"
        os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [21]:
# ============================================================================
# 3-1. 토크나이저 및 모델 로드 (Special Token 동적 등록 포함)
# ============================================================================

def load_tokenizer_and_model_for_train(config, device, verbose=True):
    """
    학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
    
    3-1 개선사항:
    - 학습 데이터에서 동적으로 수집한 special tokens 등록
    - model.resize_token_embeddings() 호출로 임베딩 사이즈 조정
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model', '-' * 10)
        print('-' * 10, f'Model Name : {config["general"]["model_name"]}', '-' * 10)
    
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(
        config['general']['model_name'], 
        config=bart_config
    )

    # ============================================================
    # 3-1. Special Tokens 등록 (동적으로 수집된 토큰 사용)
    # ============================================================
    special_tokens = config['tokenizer']['special_tokens']
    
    if verbose:
        print(f"\n📌 등록할 Special Tokens ({len(special_tokens)}개):")
        print(f"   {special_tokens}")
    
    # additional_special_tokens로 등록
    special_tokens_dict = {'additional_special_tokens': special_tokens}
    num_added = tokenizer.add_special_tokens(special_tokens_dict)
    
    if verbose:
        print(f"\n✅ 새로 추가된 토큰 수: {num_added}")
        print(f"   토크나이저 vocab size: {len(tokenizer)}")

    # ⚠️ 중요: 모델 임베딩 사이즈 조정
    generate_model.resize_token_embeddings(len(tokenizer))
    
    if verbose:
        print(f"   모델 임베딩 사이즈 조정 완료: {generate_model.config.vocab_size}")
    
    generate_model.to(device)
    
    if verbose:
        print("\n" + "-" * 60)
        print("🔍 Special Token 확인:")
        print("-" * 60)
        for token in special_tokens[:5]:  # 처음 5개만 출력
            token_id = tokenizer.convert_tokens_to_ids(token)
            print(f"   {token} → ID: {token_id}")
        if len(special_tokens) > 5:
            print(f"   ... 외 {len(special_tokens) - 5}개")
        
        print('-' * 10, 'Load tokenizer & model complete', '-' * 10)
    
    return generate_model, tokenizer


def load_tokenizer_and_model_for_inference(config, checkpoint_path, device, verbose=True):
    """
    추론을 위한 tokenizer와 fine-tuned 모델을 불러옵니다.
    
    3-1 개선사항:
    - checkpoint에서 로드 시에도 special tokens 일관성 유지
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model for inference', '-' * 10)
        print('-' * 10, f'Checkpoint: {checkpoint_path}', '-' * 10)
    
    # Checkpoint에서 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    
    # Special tokens 확인 및 추가 (checkpoint에 없을 경우)
    special_tokens = config['tokenizer']['special_tokens']
    existing_special = tokenizer.additional_special_tokens or []
    
    missing_tokens = [t for t in special_tokens if t not in existing_special]
    if missing_tokens:
        if verbose:
            print(f"⚠️ 누락된 special tokens 추가: {missing_tokens}")
        special_tokens_dict = {'additional_special_tokens': special_tokens}
        tokenizer.add_special_tokens(special_tokens_dict)
    
    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(checkpoint_path)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.eval()
    
    if verbose:
        print(f"✅ 모델 로드 완료")
        print(f"   토크나이저 vocab size: {len(tokenizer)}")
        print(f"   모델 임베딩 사이즈: {model.config.vocab_size}")
        print('-' * 10, 'Load complete', '-' * 10)
    
    return model, tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [22]:
def main(config):
    """
    모델 학습 메인 함수
    3-1, 3-2, 4, 5-1, 6-1, 6-2 개선: 
    - Special tokens + 프롬프트 + 구어체 전처리 + TF-IDF 키워드
    - 화자 수 명시 + PII 마스킹 유지 지시
    """
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
    print('-' * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-' * 10)

    # 3-2. 프롬프트 설정
    prompt_config = config.get('prompt', {
        "use_prompt": False,
        "prompt_style": "none",
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,
    })
    
    # 4. 구어체 전처리 설정
    preprocess_config = config.get('preprocess', {
        "normalize_slang": False,
    })
    
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    })
    
    print(f"\n📋 프롬프트 설정:")
    print(f"   - 프롬프트 사용: {prompt_config.get('use_prompt', False)}")
    print(f"   - 프롬프트 스타일: {prompt_config.get('prompt_style', 'none')}")
    print(f"   - 발화 구분자 사용: {prompt_config.get('use_turn_separator', False)}")
    if prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']:
        print(f"   - TF-IDF 키워드 수: {prompt_config.get('tfidf_top_k', 5)}개")
    
    print(f"\n🗣️ 구어체 전처리 설정:")
    print(f"   - 슬랭 정규화: {preprocess_config.get('normalize_slang', False)}")
    
    print(f"\n🔍 6. 정규표현식 패턴 활용:")
    print(f"   - 6-1. 화자 수 명시: {regex_pattern_config.get('use_speaker_count', True)}")
    print(f"   - 6-2. PII 마스킹 유지 지시: {regex_pattern_config.get('use_pii_instruction', True)}")
    
    # 5-1. TF-IDF extractor 사용 여부 확인
    use_tfidf = prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']
    tfidf_ext = tfidf_extractor if use_tfidf else None
    
    if use_tfidf:
        print(f"\n📊 TF-IDF 키워드 추출 활성화")
    
    preprocessor = Preprocess(
        config['tokenizer']['bos_token'], 
        config['tokenizer']['eos_token'],
        prompt_config=prompt_config,
        preprocess_config=preprocess_config,
        tfidf_extractor=tfidf_ext,           # 5-1. TF-IDF extractor 전달
        regex_pattern_config=regex_pattern_config  # 6. 정규표현식 패턴 설정 전달
    )
    
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) wandb 사용 시에만 종료
    if config['training']['report_to'] == 'wandb':
        wandb.finish()

In [23]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



📌 등록할 Special Tokens (22개):
   ['#Address#', '#Alex#', '#Bob#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#Kristin#', '#Liliana#', '#Name#', '#PassportNumber#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PersonName#', '#PhoneNumber#', '#Price#', '#SSN#']

✅ 새로 추가된 토큰 수: 22
   토크나이저 vocab size: 30022
   모델 임베딩 사이즈 조정 완료: 30022

------------------------------------------------------------
🔍 Special Token 확인:
------------------------------------------------------------
   #Address# → ID: 30002
   #Alex# → ID: 30007
   #Bob# → ID: 30008
   #CarNumber# → ID: 30006
   #CardNumber# → ID: 30020
   ... 외 17개
---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': ['#Email#', '#Name#', '#Address#', '#Person7#', '#Person3#', '#PhoneNumber#', '#CarNumber#',

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


---------- Make dataset complete ----------

📊 토큰화 결과 확인:
   첫 50 토큰 디코딩: 다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요. 

#Person1# : 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? ...
---------- Make training arguments ----------


...

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.825000,0.566371,0.344045,0.109752,0.325590
2,0.570300,0.525475,0.363801,0.128972,0.340726
3,0.495200,0.510620,0.366163,0.130008,0.344374
4,0.430900,0.509380,0.369940,0.139470,0.345888
5,0.378400,0.517669,0.366900,0.136874,0.344868
6,0.334300,0.528289,0.373542,0.140957,0.351235
7,0.295500,0.543136,0.371000,0.140288,0.347263


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 감기에 걸렸으며, 천식 검사를 위해 폐 전문의를 방문하라고 권장합니다.                                                                            
GOLD: #Person2# 는 숨쉬기 어려워합니다. 의사는 #Person2# 에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                                     
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 야 Jimmy에게 운동하러 가자고 제안합니다. 야는 토요일에 운동할 예정입니다.                                                                             
GOLD: #Person1# 는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.                                                                                
---------------------------------------------------------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/loss,█▃▁▁▂▃▅
eval/rouge-1,▁▆▆▇▆█▇
eval/rouge-2,▁▅▆█▇██
eval/rouge-l,▁▅▆▇▆█▇
eval/runtime,▄▁▄▄▂▃█
eval/samples_per_second,▅█▅▅▇▆▁
eval/steps_per_second,▅█▅▅▇▆▁
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,▁███▇▆▆
train/loss,█▂▂▁▁▁▁


## 4. 모델 추론하기

In [24]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "./prediction_kobart_v1/checkpoint-4674"

- test data를 사용하여 모델의 성능을 확인합니다.

In [25]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):
    """
    3-2 개선: 프롬프트 설정이 적용된 테스트 데이터셋 준비
    """
    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'[원본] test_data dialogue:\n{test_data["dialogue"][0][:200]}...')
    print('-'*150)

    # 3-2. 프롬프트 적용된 테스트 데이터로 변환
    encoder_input_test, decoder_input_test = preprocessor.make_input(
        test_data, 
        is_test=True,
        apply_dynamic_prompt=True  # 프롬프트 설정 사용
    )
    
    # 프롬프트 적용 확인
    print(f'[프롬프트 적용 후] encoder_input_test[0]:\n{encoder_input_test[0][:300]}...')
    print('-'*10, 'Load data complete', '-'*10)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10)

    return test_data, test_encoder_inputs_dataset

In [26]:
# ============================================================================
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
# 3-1 개선: 동적으로 수집된 special tokens 사용
# ============================================================================

def load_tokenizer_and_model_for_test(config, device, verbose=True):
    """
    추론용 토크나이저 및 모델 로드
    3-1: 동적으로 수집된 special tokens 등록
    """
    if verbose:
        print('-' * 10, 'Load tokenizer & model', '-' * 10)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    
    if verbose:
        print('-' * 10, f'Model Name : {model_name}', '-' * 10)
        print('-' * 10, f'Checkpoint : {ckt_path}', '-' * 10)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # 3-1. 동적으로 수집된 special tokens 등록
    special_tokens = config['tokenizer']['special_tokens']
    special_tokens_dict = {'additional_special_tokens': special_tokens}
    num_added = tokenizer.add_special_tokens(special_tokens_dict)
    
    if verbose:
        print(f"\n📌 등록된 Special Tokens ({len(special_tokens)}개)")
        print(f"   새로 추가된 토큰: {num_added}개")

    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    
    if verbose:
        print(f"   토크나이저 vocab size: {len(tokenizer)}")
        print(f"   모델 임베딩 사이즈: {generate_model.config.vocab_size}")
        print('-' * 10, 'Load tokenizer & model complete', '-' * 10)

    return generate_model, tokenizer

In [27]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    """
    모델 추론 함수
    3-1, 3-2, 4, 5-1, 5-2, 6-1, 6-2 개선: 
    - Special tokens 동적 등록
    - 프롬프트 설정
    - 구어체 전처리 적용
    - TF-IDF 키워드 프롬프트
    - 동적 추론 설정 (길이 기반)
    - 화자 수 명시 (6-1)
    - PII 마스킹 유지 지시 (6-2)
    """
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 3-2. 프롬프트 설정
    prompt_config = config.get('prompt', {
        "use_prompt": False,
        "prompt_style": "none",
        "use_turn_separator": False,
        "turn_separator": "[SEP]",
        "tfidf_top_k": 5,
    })
    
    # 4. 구어체 전처리 설정
    preprocess_config = config.get('preprocess', {
        "normalize_slang": False,
    })
    
    # 6. 정규표현식 패턴 활용 설정 (NEW)
    regex_pattern_config = config.get('regex_pattern', {
        "use_speaker_count": True,       # 6-1. 화자 수 명시
        "use_pii_instruction": True,     # 6-2. PII 마스킹 유지 지시
        "show_pii_examples": True,       # 마스킹 예시 표시
    })
    
    # 5-2. 동적 추론 설정 (길이 기반)
    dynamic_inference_config = config.get('dynamic_inference', {})
    use_dynamic = dynamic_inference_config.get('use_length_based', False) or dynamic_inference_config.get('use_dynamic_settings', False)
    
    print(f"\n📋 추론 시 프롬프트 설정:")
    print(f"   - 프롬프트 사용: {prompt_config.get('use_prompt', False)}")
    print(f"   - 프롬프트 스타일: {prompt_config.get('prompt_style', 'none')}")
    if prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']:
        print(f"   - TF-IDF 키워드 수: {prompt_config.get('tfidf_top_k', 5)}개")
    
    print(f"\n🗣️ 추론 시 구어체 전처리 설정:")
    print(f"   - 슬랭 정규화: {preprocess_config.get('normalize_slang', False)}")
    
    print(f"\n🔍 6. 정규표현식 패턴 활용:")
    print(f"   - 6-1. 화자 수 명시: {regex_pattern_config.get('use_speaker_count', True)}")
    print(f"   - 6-2. PII 마스킹 유지 지시: {regex_pattern_config.get('use_pii_instruction', True)}")
    
    print(f"\n⚙️ 동적 추론 설정:")
    print(f"   - 동적 설정 사용: {use_dynamic}")
    if use_dynamic:
        print(f"   - 길이 기반: {dynamic_inference_config.get('use_length_based', False)}")
        print(f"   - 주제 기반: {dynamic_inference_config.get('use_topic_based', False)}")
    
    # 5-1. TF-IDF extractor 사용 여부 확인
    use_tfidf = prompt_config.get('prompt_style') in ['keyword', 'keyword_aware']
    tfidf_ext = tfidf_extractor if use_tfidf else None
    
    if use_tfidf:
        print(f"\n📊 TF-IDF 키워드 추출 활성화")
    
    preprocessor = Preprocess(
        config['tokenizer']['bos_token'], 
        config['tokenizer']['eos_token'],
        prompt_config=prompt_config,               # 3-2. 프롬프트 설정 전달
        preprocess_config=preprocess_config,       # 4. 구어체 전처리 설정 전달
        tfidf_extractor=tfidf_ext,                 # 5-1. TF-IDF extractor 전달
        regex_pattern_config=regex_pattern_config  # 6. 정규표현식 패턴 설정 전달
    )

    data_path = config['general']['data_path']
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    
    # 5-2. 동적 추론을 위해 원본 텍스트 길이 정보 필요
    dialogue_lengths = []
    if use_dynamic:
        # test_data에서 대화문 길이 계산
        for idx in range(len(test_data)):
            dialogue = test_data.iloc[idx]['dialogue']
            dialogue_lengths.append(len(dialogue))
        print(f"\n📏 대화문 길이 분포: min={min(dialogue_lengths)}, max={max(dialogue_lengths)}, avg={sum(dialogue_lengths)/len(dialogue_lengths):.0f}")
    
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    
    # 5-2. 동적 추론 시 배치 단위가 아닌 개별 샘플 처리 필요
    if use_dynamic:
        print("\n🚀 동적 추론 모드로 생성 시작...")
        sample_idx = 0
        use_topic = dynamic_inference_config.get('use_topic_based', True)
        use_length = dynamic_inference_config.get('use_length_based', True)
        
        with torch.no_grad():
            for item in tqdm(dataloader):
                batch_size = len(item['ID'])
                text_ids.extend(item['ID'])
                
                for i in range(batch_size):
                    # 개별 샘플에 대한 동적 설정 계산 (원본 대화문 사용)
                    dialogue_text = test_data.iloc[sample_idx]['dialogue']
                    dynamic_cfg = get_dynamic_inference_config(dialogue_text, use_topic=use_topic, use_length=use_length)
                    
                    # 단일 샘플 생성
                    single_input_ids = item['input_ids'][i:i+1].to(device)
                    generated_ids = generate_model.generate(
                        input_ids=single_input_ids,
                        no_repeat_ngram_size=dynamic_cfg['no_repeat_ngram_size'],
                        early_stopping=config['inference']['early_stopping'],
                        max_length=dynamic_cfg['max_new_tokens'],
                        num_beams=dynamic_cfg['num_beams'],
                        length_penalty=dynamic_cfg['length_penalty'],
                    )
                    
                    # 3-1 개선: 화자 태그 보존하면서 디코딩
                    result = decode_with_speaker_tags(tokenizer, generated_ids[0])
                    summary.append(result)
                    sample_idx += 1
    else:
        # 기존 배치 처리 방식
        with torch.no_grad():
            for item in tqdm(dataloader):
                text_ids.extend(item['ID'])
                generated_ids = generate_model.generate(
                    input_ids=item['input_ids'].to(device),
                    no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                    early_stopping=config['inference']['early_stopping'],
                    max_length=config['inference']['generate_max_length'],
                    num_beams=config['inference']['num_beams'],
                )
                for ids in generated_ids:
                    # 3-1 개선: 화자 태그 보존하면서 디코딩
                    result = decode_with_speaker_tags(tokenizer, ids)
                    summary.append(result)

    # 후처리 적용 (띄어쓰기 교정)
    preprocessed_summary = [postprocess_summary(s) for s in summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary": preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [30]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------
---------- Checkpoint : ./prediction_kobart_v1/checkpoint-4674 ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



📌 등록된 Special Tokens (22개)
   새로 추가된 토큰: 22개
   토크나이저 vocab size: 30022
   모델 임베딩 사이즈: 30022
---------- Load tokenizer & model complete ----------

📋 추론 시 프롬프트 설정:
   - 프롬프트 사용: True
   - 프롬프트 스타일: balanced

🗣️ 추론 시 구어체 전처리 설정:
   - 슬랭 정규화: True

🔍 6. 정규표현식 패턴 활용:
   - 6-1. 화자 수 명시: True
   - 6-2. PII 마스킹 유지 지시: True

⚙️ 동적 추론 설정:
   - 동적 설정 사용: True
   - 길이 기반: True
   - 주제 기반: True
------------------------------------------------------------------------------------------------------------------------------------------------------
[원본] test_data dialogue:
#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. 
#Person2#: 네, 말씀하세요... 
#Person1#: 이걸 오늘 오후까지 모든 직원들에게 사내 메모로 보내야 해요. 준비됐나요? 
#Person2#: 네, 말씀하세요. 
#Person1#: 모든 직원에게 알립니다... 즉시 발효되어 모든 사내 통신은 이메일과 공식 메모로만 제...
------------------------------------------------------------------------------------------------------------------------------------------------------
[프롬프트 적용 후] encoder_input_test[0]:
다음은 2명이 나눈 대화입니다. 두 사람의 입장을 균형 있게 반영해서 요약해 주세요

100%|██████████| 16/16 [01:24<00:00,  5.29s/it]



In [31]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,#Person1# 은 Ms. Dawson에게 이메일과 공식 메모로만 제한되는 사내 ...
1,test_1,#Person1# 과 #Person2# 는 교통체증으로 인해 출퇴근에 어려움을 겪고...
2,test_2,#Person1# 은 Kate에게 Masha와 Hero가 두 달 동안 별거하다가 결...
3,test_3,#Person1# 은 Brian에게 생일 파티에서 함께 춤을 추자고 초대합니다.
4,test_4,#Person1# 과 #Person2# 는 올림픽 공원의 크기와 시설에 대해 이야기...
...,...,...
494,test_495,"Jack은 Charlie에게 학교 후 집에서 그녀와 비디오 게임을 하자고 제안하고,..."
495,test_496,#Person2# 는 시골 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하게...
496,test_497,Alice는 #Person1# 에게 세탁기에 비누가 들어 있지 않아 따로 넣어야 한...
497,test_498,"Matthew는 임대 계약을 갱신하고 싶지 않아 집을 찾고 있으며, Mrs. Tho..."
